##  Gestión de Tiendas: diseño y poblado de una BBDD SQL con Python 
# MERCAZONA 

Este proyecto desarrolla una base de datos relacional para la gestión de una cadena de tiendas, utilizando **MySQL Workbench** para la estructuración del esquema y **Python** para la automatización de la carga de datos. A lo largo del proceso, se explican los pasos: desde la creación del esquema SQL hasta la inserción de datos mediante programación **Python**.

El modelo de datos está compuesto por siete tablas clave, representando entidades como tiendas, empleados, productos, clientes y órdenes de compra. Los datos se generan de forma programada y aleatoria, manteniendo una distribución realista. Los números y distribución son los siguientes:

* **10 tiendas** con **200 empleados**, distribuidos en diferentes roles.
* **40 productos**, organizados en **10 categorías**.
* **2000 clientes**, con una distribución no uniforme, priorizando la región andaluza territorio de origen de la marca.
* **10,000 órdenes de compra**, cada una con múltiples artículos, sumando **30,000 detalles de orden**.

El código que se ha realizado en Python automatiza la generación y el envío de estos datos a la base de datos MySQL, asegurando una integración eficiente y reproducible.

Este repositorio ademas de servir como un portafolio técnico, también puede ser un recurso educativo para quienes buscan aprender sobre bases de datos relacionales, modelado de datos y automatización con Python. 

## Empecemos: 

### 1️⃣ **Instalación del conector de MySQL**
Antes de ejecutar el código, se instala la librería :       
(Permite la conexión con la base de datos MySQL.)

In [17]:
%pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



### 2️⃣ **Código completo de envío del esquema SQL**
El siguiente código en Python realiza la lectura del SQL que contiene el esquema de la BBDD, establece una conexión con MySQL y ejecuta las sentencias contenidas en el archivo. Para el manejo de errores se introduce laestructura try-exept-fynally, cerrando la conexión en último lugar.


In [18]:
import mysql.connector as con

try:
    with open('MercaZona_schema2.sql', mode='r', encoding='utf-8') as file:
        sql = file.read()
    
    connection = con.connect(
        host="localhost",
        port="3306",
        user="root",
        password="admin"
    )
    cursor = connection.cursor()
    for statement in sql.split(';'):
        if statement.strip():
            cursor.execute(statement)
    print("TODO OK.")

except con.Error as e:
    print(f"Error al interactuar con la base de datos: {e}")
    if connection:
        connection.rollback()

finally:
    if 'cursor' in locals() and cursor:
        cursor.close()
    if 'connection' in locals() and connection.is_connected():
        connection.close()
    print("Recursos cerrados correctamente.")



TODO OK.
Recursos cerrados correctamente.


##### **Explicación del Código**

1. **Lectura del archivo SQL**: Se abre y se lee el contenido del archivo `MercaZona_schema2.sql` (que se adjunta al proyecto) en modo de lectura (`'r'`) con codificación UTF-8.

2. **Conexión a MySQL**: Parámetros:
   - `host="localhost"`: Indica que el servidor de la base de datos se ejecuta en la misma máquina.
   - `port="3306"`: Especifica el puerto predeterminado en el que MySQL está escuchando conexiones.
   
3. **Ejecución de sentencias SQL**: Se divide el contenido del archivo en sentencias individuales utilizando `split(';')`. Luego, se recorre cada una de ellas y se ejecuta con `cursor.execute(statement)`, asegurando que todas las columnas se procesen correctamente.
4. **Manejo de errores**: Se implementa un bloque `try-except` para capturar errores de conexión o ejecución de SQL. En caso de fallo, se muestra un mensaje con la descripción del error y se revierte cualquier operación pendiente con `connection.rollback()`.
5. **Cierre de conexión**: Se verifica que si los objetos `cursor` y `connection` existen y están activos se cierren correctamente para evitar problemas.


### 3️⃣ **Creación del DataFrame de Tiendas**
El siguiente código crea un `DataFrame` con la información de las tiendas, incluyendo su nombre, dirección y ciudad.

In [19]:
import pandas as pd

tiendas = ['MZ_Madrid', 'MZ_Barcelona', 'MZ_Vigo', 'MZ_Sevilla', 'MZ_Bilbao', 'MZ_Valencia', 'MZ_Málaga', 'MZ_Toledo', 'MZ_Cádiz', 'MZ_Murcia']
direccion = ['Calle del Llano, 40', 'Carrer Splugat del Ferrer, 4', 'Rua da Trapallada , 20', 'Calle del Sobrao, 15', 'Harrizko kalea, 21', 'Carrer Sort, 23', 'Calle del Espeto, 2', 'Paseo Imperial, 43', 'Calle del Quillo, 6', 'Calle Queloqué, 1']
ciudad = ['Madrid', 'Barcelona', 'Vigo', 'Sevilla', 'Bilbao', 'Valencia', 'Málaga', 'Toledo', 'Cádiz', 'Murcia']

df_tiendas = pd.DataFrame(
    zip(tiendas, direccion, ciudad),
    columns=['nombre_tienda', 'direccion', 'ciudad']
)

df_tiendas = df_tiendas.reset_index().rename({'index': 'id_tienda'}, axis=1)
df_tiendas['id_tienda'] = df_tiendas['id_tienda'] + 1
df_tiendas.head()

,id_tienda,nombre_tienda,direccion,ciudad
0,1,MZ_Madrid,"Calle del Llano, 40",Madrid
1,2,MZ_Barcelona,"Carrer Splugat del Ferrer, 4",Barcelona
2,3,MZ_Vigo,"Rua da Trapallada , 20",Vigo
3,4,MZ_Sevilla,"Calle del Sobrao, 15",Sevilla
4,5,MZ_Bilbao,"Harrizko kalea, 21",Bilbao


##### **Explicación del Código**

1. **Definición de listas**: Se crean tres listas (`tiendas`, `direccion` y `ciudad`), para posteriormente agregarlas como contenido de las respectivas columnas..
2. **Creación del DataFrame**:
   - Se genera un `DataFrame` con `pd.DataFrame`.
   - Se usa `zip()` para combinar las listas en una estructura de columnas.
3. **Añadir una columna de ID**:
   - Se usa `.reset_index()` para agregar un índice numérico y se renombra como `id_tienda`.
   - Se ajusta el índice sumando `+1` para que empiece desde 1 en lugar de 0.

Este DataFrame será la base para cargar información de tiendas en la base de datos MySQL.



### 4️⃣ **Creación del DataFrame de Empleados**
Este código automatiza la generación de datos para la base de datos de tiendas. Incluye los empleados con una distribución por tienda y asignación de puestos de trabajo estructurada.

In [20]:
puestos_por_tienda = {
    'Gerente': 1,
    'Encargada/o Tienda': 1,
    'Responsable Almacén': 1,
    'Atención al Cliente': 3,
    'Limpiador/a': 3,
    'Reponedor/a': 5,
    'Cajero/a': 6 }

empleados_x_tienda = sum(puestos_por_tienda.values())
total_empleados = len(df_tiendas) * empleados_x_tienda

empleados = []
contador = 1

for id_tienda in df_tiendas['id_tienda']:
    for _ in range(empleados_x_tienda):
        empleados.append({'empleado': f'Emplead@{contador}', 'id_tienda': id_tienda})
        contador += 1

puestos = []
for id_tienda in df_tiendas['id_tienda']:
    for puesto, cantidad in puestos_por_tienda.items():
        for _ in range(cantidad):
            puestos.append({'puesto': puesto, 'id_tienda': id_tienda})

df_empleados = pd.DataFrame(
    zip([emp['empleado'] for emp in empleados], [puesto['puesto'] for puesto in puestos], [emp['id_tienda'] for emp in empleados]),
    columns=['nombre_empleado', 'puesto', 'id_tienda'])

df_empleados = df_empleados.reset_index().rename({'index': 'id_empleado'}, axis=1)
df_empleados['id_empleado'] = df_empleados['id_empleado'] + 1
df_empleados.head()

,id_empleado,nombre_empleado,puesto,id_tienda
0,1,Emplead@1,Gerente,1
1,2,Emplead@2,Encargada/o Tienda,1
2,3,Emplead@3,Responsable Almacén,1
3,4,Emplead@4,Atención al Cliente,1
4,5,Emplead@5,Atención al Cliente,1


In [21]:
print(f'El número total de empleados en la cadena de supermercados MercaZona es de {total_empleados} empleados.')
print(f'El número de empleados por tienda es de {empleados_x_tienda} empleados.') 
print(f'Los puestos están dstribuidos de la siguiente manera:') 
print(f'{puestos_por_tienda}')

El número total de empleados en la cadena de supermercados MercaZona es de 200 empleados.
El número de empleados por tienda es de 20 empleados.
Los puestos están dstribuidos de la siguiente manera:
{'Gerente': 1, 'Encargada/o Tienda': 1, 'Responsable Almacén': 1, 'Atención al Cliente': 3, 'Limpiador/a': 3, 'Reponedor/a': 5, 'Cajero/a': 6}


##### **Explicación del Código**

1. **Definición de puestos**: Se crea un diccionario con la cantidad de cada puesto por tienda.
2. **Cálculo del total de empleados**: Se multiplica la cantidad de empleados por tienda por el número total de tiendas.
3. **Generación de empleados**:
   - Se asigna un identificador único a cada empleado y se asocia con su tienda correspondiente.
4. **Asignación de puestos**:
   - Se generan los puestos en función de la cantidad definida en el diccionario y se asocian con cada tienda.
5. **Creación del DataFrame**:
   - Se combinan los datos de empleados y puestos en un `DataFrame` usando .zip y una iteracion que recorra todos los datos las columnas. 
   - Se agrega un `id_empleado`.
Este DataFrame servirá como base para la carga de datos en la tabla de empleados de la base de datos MySQL.

### 5️⃣ **Creación del DataFrame de Categorías**
Este código realiza la cración de categorias para la base de datos de tiendas, creando las mismas de manera estructurada y a ellas luego se añadiran los produtos.

In [22]:
categorias = [
    "Alimentación Fresca",
    "Alimentación No Perecedera",
    "Platos Preparados",
    "Bebidas",
    "Congelados",
    "Panadería y Repostería",
    "Productos de Higiene",
    "Droguería y Limpieza",
    "Bebé y Puericultura",
    "Mascotas"
]

df_categoria = pd.DataFrame(categorias, columns=['nombre_categoria'])

df_categoria = df_categoria.reset_index().rename({'index': 'id_categoria'}, axis=1)
df_categoria['id_categoria'] = df_categoria['id_categoria'] + 1
df_categoria.head(11)

,id_categoria,nombre_categoria
0,1,Alimentación Fresca
1,2,Alimentación No Perecedera
2,3,Platos Preparados
3,4,Bebidas
4,5,Congelados
5,6,Panadería y Repostería
6,7,Productos de Higiene
7,8,Droguería y Limpieza
8,9,Bebé y Puericultura
9,10,Mascotas


##### **Explicación del Código**

1. **Definición de categorías**: Se crea una lista con los diferentes tipos de categorías de productos disponibles en la tienda.
2. **Creación del DataFrame**:
   - Se usa `pd.DataFrame()` para transformar la lista en un DataFrame con una sola columna llamada `nombre_categoria`.
   - Se genera un índice numérico que se renombra como `id_categoria`.
   - Se ajusta la numeración para que empiece desde 1 sumando `+1` a cada índice, como en anteriores ocasiones.

Este DataFrame servirá como base para la carga de datos en la tabla de categorías de la base de datos MySQL.

### 6️⃣ **Creación del DataFrame de Productos** 
Dataset de creación de productos disponibles en cada categoría, con sus respectivos precios y stock aleatorio.

In [23]:
import random

productos_X_cat = {
    "Alimentación Fresca": ["Manzana", "Lechuga", "Pollo", "Gambones"],
    "Alimentación No Perecedera": ["Arroz", "Pasta", "Lentejas", "Aceite"],
    "Platos Preparados": ["Lasagna", "Pizza", "Ensaladilla", "Canelones"],
    "Bebidas": ["Agua", "Jugo", "Refresco", "Cerveza"],
    "Congelados": ["Helado", "Guisantes", "Medallones", "San Jacobos"],
    "Panadería y Repostería": ["Pan Integral", "Croissant", "Magdalena", "Rosquillas"],
    "Productos de Higiene": ["Jabón", "Champú", "Desodorante", "Pasta de dientes"],
    "Droguería y Limpieza": ["Detergente", "Lejía", "Suavizante", "Sanitol"],
    "Mascotas": ["Purina", "Advance", "Correa", "Arena"]
}

precios_X_producto = {producto: random.uniform(0.50, 10.00) for categoria in productos_X_cat.values() for producto in categoria}

productos_datos = []
for id_categoria, (categoria, productos) in zip(df_categoria['id_categoria'], productos_X_cat.items()):
    for producto in productos:
        productos_datos.append({
            'nombre_producto': producto,
            'precio': precios_X_producto[producto],
            'stock': random.randint(0, 500),
            'id_categoria': id_categoria
        })

df_productos = pd.DataFrame(productos_datos)
df_productos = df_productos.reset_index().rename({'index': 'id_producto'}, axis=1)
df_productos['id_producto'] = df_productos['id_producto'] + 1
df_productos.head()

,id_producto,nombre_producto,precio,stock,id_categoria
0,1,Manzana,0.787383,53,1
1,2,Lechuga,2.054171,216,1
2,3,Pollo,4.753901,251,1
3,4,Gambones,3.619864,147,1
4,5,Arroz,5.313727,59,2


##### **Explicación del Código**

1. **Definición de productos**: Se establece un diccionario con categorías de productos y una lista de productos por cada una.
2. **Asignación de precios y stock**:
   - Se generan precios aleatorios para cada producto.
   - Se genera una cantidad de stock aleatoria entre 0 y 500 unidades.
3. **Creación del DataFrame**:
   - Se combinan los datos de productos, precios y stock en un `DataFrame`.
   - Se genera un índice único para cada producto, comenzando desde `1`.
Este DataFrame servirá como base para la carga de datos en la tabla de productos de la base de datos MySQL.


### 7️⃣ **Creación del DataFrame de Clientes**
Se genera un conjunto de datos que incluye nombres, apellidos, correos electrónicos, teléfonos, codigo postal y ubicación asignada a una tienda.

In [24]:
import random

random.seed(2021)
n_filas = 2000

nombres = [f'cliente0{numero}' if numero < 10 else f'Cliente{numero}' for numero in range(n_filas)]
apellidos = [f'apellido0{numero}' if numero < 10 else f'apellido{numero}' for numero in range(n_filas)]

def generar_telefono():
    return f"6{random.randint(10000000, 99999999)}"

telefono = [generar_telefono() for _ in range(n_filas)]
emails = [f'{nombre}.{apellido}@jemail.com' for nombre, apellido in zip(nombres, apellidos)]

codigo_postal_prefix = {
    'Madrid': 28,
    'Barcelona': 8,
    'Vigo': 36,
    'Sevilla': 41,
    'Bilbao': 48,
    'Valencia': 46,
    'Málaga': 29,
    'Toledo': 45,
    'Cádiz': 11,
    'Murcia': 30
}

pesos = [0.11, 0.09, 0.08, 0.15, 0.08, 0.04, 0.14, 0.03, 0.20, 0.05]
pesos_nor = [p / sum(pesos) for p in pesos]  

id_tienda = random.choices(df_tiendas['id_tienda'], weights=pesos_nor, k=n_filas)

df_clientes = pd.DataFrame(
    zip(nombres, apellidos, emails, telefono, id_tienda),
    columns=['first_name', 'last_name', 'email', 'telefono', 'id_tienda']
)

df_clientes = df_clientes.merge(df_tiendas, on='id_tienda', how='left')
df_clientes['codigo_postal'] = df_clientes['ciudad'].map(lambda x: f"{codigo_postal_prefix[x]:02}{random.randint(100, 999)}")

df_clientes.drop(columns=['nombre_tienda', 'ciudad', 'direccion'], inplace=True)
df_clientes = df_clientes.reset_index().rename({'index': 'id_cliente'}, axis=1)
df_clientes['id_cliente'] = df_clientes['id_cliente'] + 1

df_clientes.head()

,id_cliente,first_name,last_name,email,telefono,id_tienda,codigo_postal
0,1,cliente00,apellido00,cliente00.apellido00@jemail.com,664251571,7,29259
1,2,cliente01,apellido01,cliente01.apellido01@jemail.com,694525032,9,11100
2,3,cliente02,apellido02,cliente02.apellido02@jemail.com,683029252,9,11323
3,4,cliente03,apellido03,cliente03.apellido03@jemail.com,647139583,9,11876
4,5,cliente04,apellido04,cliente04.apellido04@jemail.com,643196383,7,29737


+ Numero de clientes por tienda. 

In [25]:
clientes_por_tienda = df_clientes['id_tienda'].value_counts().reset_index()
clientes_por_tienda.columns = ['tienda_id', 'num_clientes']

clientes_X_tienda = clientes_por_tienda.sort_values(by='num_clientes', ascending=False)
clientes_X_tienda

,tienda_id,num_clientes
0,9,404
1,4,330
2,7,286
3,1,211
4,2,183
5,3,166
6,5,158
7,10,116
8,6,81
9,8,65


##### **Explicación del Código**

En este bloque de código, generamos un total de **2000 clientes ficticios**, cada uno con su nombre, apellido, código postal, correo electrónico y número de teléfono. Para darle un toque de realismo, los clientes no se distribuyen de manera uniforme en las tiendas, sino que se priorizan ciertas regiones por el origuen y antiguedad del establecimiento en ellas. En particular, las tiendas donde nació la marca cuentan con una mayor afluencia de clientes en comparación con las tiendas de otras provincias."

Dado que la empresa tiene sus raíces en Andalucía, hay una mayor cantidad de clientes registrados en tiendas de esta comunidad. Así que si ves que hay más clientes en Sevilla, Málaga o Cádiz, no es un bug, es un feature.
(Vale, sí, un poco geek, seguimos...😅). Esta distribución desigual se decide a través de los **pesos** asignados a cada provincia.

Para completar la información del cliente, se generan códigos postales que reflejan su ubicación en función de la tienda asignada. Cada código sigue un patrón de prefijos geográficos reales, añadiendo un número aleatorio al final para simular una dirección más auténtica. 

Finalmente, los datos se integran en un `DataFrame`, eliminando columnas innecesarias y organizando los registros para su posterior inserción en la base de datos.

### 8️⃣ **Creación del DataFrame de Órdenes o Tickets** 

Las órdenes o tickets de compra representan las transacciones realizadas por los clientes en las tiendas. Cada orden tiene una fecha y hora de compra, está asociada a un cajero específico y puede ser pagada en **efectivo, tarjeta o vale**. Además, para simular un entorno más realista, se excluyen los **domingos** y fechas festivas importantes, evitando que se generen ventas en días no laborables.

In [26]:
import random
from datetime import datetime, timedelta

num_ordenes = 10000
random.seed(2021)

id_tienda = df_tiendas['id_tienda'].unique().tolist()
id_cliente = df_clientes['id_cliente'].tolist()

id_tienda = random.choices(id_tienda, k=num_ordenes)
id_cliente = random.choices(id_cliente, k=num_ordenes)

id_cajeros = df_empleados[df_empleados['puesto'] == 'Cajero/a']
id_cajero = id_cajeros['id_empleado'].tolist()

cajeros_por_tienda = {t: id_cajeros[id_cajeros['id_tienda'] == t]['id_empleado'].tolist() for t in id_tienda}
cajero_id = [random.choice(cajeros_por_tienda[tienda]) for tienda in id_tienda]

dias_excluidos = {(1, 1), (6, 1), (1, 5), (6, 12), (8, 12), (25, 12)}
dias_semana_excluidos = {6}  # Excluir domingos

fecha_orden = []
while len(fecha_orden) < num_ordenes:
    fecha = datetime.now().date() - timedelta(days=random.randint(0, 365))
    if (fecha.month, fecha.day) not in dias_excluidos and fecha.weekday() not in dias_semana_excluidos:
        fecha_orden.append(fecha.strftime('%Y-%m-%d'))

hora_orden = [datetime.strptime(f"{random.randint(10, 22)}:{random.randint(0, 59)}:{random.randint(0, 59)}", "%H:%M:%S").time() for _ in range(num_ordenes)]

metodos_pago = ['tarjeta', 'efectivo', 'vale']
metodo_de_pago = random.choices(metodos_pago, k=len(fecha_orden))

df_ordenes = pd.DataFrame(
    zip(id_cliente, cajero_id, id_tienda, fecha_orden, hora_orden, metodo_de_pago),
    columns=['id_cliente', 'id_empleado', 'id_tienda', 'fecha_orden', 'hora_orden', 'metodo_de_pago']
)

df_ordenes = df_ordenes.reset_index().rename({'index': 'id_orden'}, axis=1)
df_ordenes['id_orden'] = df_ordenes['id_orden'] + 1

df_ordenes.head()

,id_orden,id_cliente,id_empleado,id_tienda,fecha_orden,hora_orden,metodo_de_pago
0,1,1938,175,9,2024-11-06,19:18:10,tarjeta
1,2,1235,178,9,2024-12-21,18:06:24,vale
2,3,1569,117,6,2024-12-19,10:56:44,vale
3,4,571,60,3,2024-08-17,16:17:47,tarjeta
4,5,1578,138,7,2024-09-13,16:34:10,efectivo


##### **Explicación del Código**

El código desarrolla diversas técnicas para garantizar una correcta simulación de órdenes de compra en el entorno de las tiendas. Utiliza las bibliotecas `random` y `datetime` para generar fechas, horas y selecciones aleatorias de clientes, empleados y tiendas que se incluyen en las órdenes.

El proceso de generación de órdenes está pensado para simular la actividad comercial real de las tiendas. Se generan **10,000 órdenes de compra**, cada una asociada a un cliente y a un cajero asignado aleatoriamente dentro de la tienda correspondiente. Esto permite reflejar transacciones de manera verosímil. Se establecen ciertas restricciones en las fechas, asegurando que no se generen órdenes en **domingos ni en días festivos**, como Año Nuevo o Navidad.

Además, cada orden se genera con una hora aleatoria dentro del rango de **10:00 a 22:00**, simulando el horario comercial de las tiendas. También se clasifica cada orden en uno de los tres métodos de pago disponibles: **tarjeta, efectivo o vale**.

Se utiliza `random.choices()` para seleccionar aleatoriamente clientes y tiendas, asegurando una distribución variada de las órdenes. La asignación de cajeros también se hace dinámicamente, filtrando solo aquellos empleados cuyo puesto es 'Cajero/a'.

Para las fechas de las órdenes, `datetime.now()` y `timedelta(days=random.randint(0, 365))` permiten seleccionar fechas aleatorias dentro del último año, excluyendo domingos y días festivos mediante estructuras de conjuntos.

El `zip()` se usa para ensamblar los datos en el `DataFrame`, combinando las listas generadas.


### 9️⃣ **Generación del Detalle de Órdenes**

El detalle de órdenes representa cada línea de productos comprados dentro de una orden o ticket. Cada ticket puede contener múltiples productos con su cantidad, precio unitario y, en algunos casos, descuentos aplicados. Para asegurar una distribución realista de los datos, los productos se asignan aleatoriamente a cada orden, y se generan descuentos ocasionales.

In [27]:
num_detalles = 30000
random.seed(2021)

id_orden = random.choices(df_ordenes['id_orden'].tolist(), k=num_detalles)
id_producto = random.choices(df_productos['id_producto'].tolist(), k=num_detalles)
cantidad = [random.randint(1, 20) for _ in range(num_detalles)]

descuento = [round(random.uniform(0.10, 2.50), 2) if random.random() < 0.85 else None 
            for _ in range(num_detalles)]

df_detalle_orden = pd.DataFrame(zip(id_orden, id_producto, cantidad, descuento),
                                columns= ['id_orden', 'id_producto', 'cantidad', 'descuento'])

df_detalle_orden = df_detalle_orden.merge(df_productos[['id_producto', 'precio']], on='id_producto', how='left')
df_detalle_orden.rename(columns={'precio':'precio_unitario'}, inplace=True)

df_detalle_orden = df_detalle_orden.reset_index().rename({'index': 'id_detalle'}, axis=1)
df_detalle_orden['id_detalle'] = df_detalle_orden['id_detalle'] + 1

df_detalle_orden.fillna({'descuento': 0}, inplace=True)
column_order = ['id_detalle','id_orden', 'id_producto', 'cantidad', 'precio_unitario', 'descuento']
df_detalle_orden = df_detalle_orden[column_order]

df_detalle_orden.head()

,id_detalle,id_orden,id_producto,cantidad,precio_unitario,descuento
0,1,8364,16,11,7.436235,2.37
1,2,8584,27,9,3.773355,0.54
2,3,5442,25,6,2.881836,1.61
3,4,2474,5,7,5.313727,0.00
4,5,6351,25,17,2.881836,1.13


##### **Explicación del Código** 

Este código genera **30,000 líneas de detalle de órdenes**, asignando productos y cantidades de manera aleatoria para cada transacción. Se utiliza `random.choices()` para seleccionar productos y órdenes existentes, asegurando una distribución equilibrada.

La cantidad de productos por línea de orden se establece mediante `random.randint(1, 20)`, generando tickets con compras de distintos tamaños. Además, se aplica un descuento aleatorio en aproximadamente el **85% de los registros**, utilizando `random.uniform(0.10, 2.50)`, lo que simula promociones y descuentos frecuentes en los productos, es decir el 85% de los productos tiene algún tipo de descuento entre 0.10 y 2.50 euros.

El código también realiza un **merge** con la tabla de productos para incluir el precio unitario de cada ítem dentro del detalle de la orden. 
Mediante `.fillna()`, se asigna un valor de 0 a los descuentos none, el 15%, asegurando que todas las lineas de descuento tengn un valor.


### 1️⃣0️⃣ **Exportación de los DataFrames a CSV**

Para facilitar la carga de los datos en la base de datos MySQL, se exportan todos los DataFrames generados a archivos CSV. Posteriormente, estos archivos podrán ser utilizados para su inserción en MySQL mediante una función que se define en la siguiente sección.

In [28]:
df_tiendas.to_csv('tiendas.csv', index=False)
df_empleados.to_csv('empleados.csv', index=False)
df_categoria.to_csv('categoria.csv', index=False)
df_productos.to_csv('productos.csv', index=False)
df_clientes.to_csv('clientes.csv', index=False)
df_ordenes.to_csv('ordenes.csv', index=False)
df_detalle_orden.to_csv('detalle_orden.csv', index=False)


##### **Explicación del Código** 

La exportación de los DataFrames a CSV se realiza utilizando el método `.to_csv()` de Pandas. Este proceso es fundamental para garantizar que los datos generados en Python puedan ser utilizados en entornos de bases de datos relacionales como MySQL.

Cada archivo CSV se genera sin incluir el índice (`index=False`) para evitar agregar una columna innecesaria que pueda generar duplicidades.

En la siguiente sección se implementará una función para insertar los datos en MySQL de manera eficiente y a continuación se llevara a cabo su inserción.

###  1️⃣1️⃣ **Inserción de Datos en MySQL**

Para poblar la base de datos con los datos generados, se define una función que permite insertar cualquier `DataFrame` en una tabla de MySQL. Esta función maneja posibles errores y utiliza una inserción eficiente con `executemany`.


In [29]:
def insertar_dataframe(df, table_name):
    try:
        connection = con.connect(
            host="localhost",
            port="3306",
            user="root",
            password="admin",
            database='mercazona'
        )
        cursor = connection.cursor()
        
        columns = ','.join(df.columns)
        placeholders = ','.join(['%s'] * len(df.columns))
        sql = f'INSERT INTO {table_name} ({columns}) VALUES ({placeholders});'
    
        rows = [tuple(row) for index, row in df.iterrows()] 
        cursor.executemany(sql, rows)
        
        connection.commit()
        return cursor.rowcount
    except con.Error as error:
        print(f"Ha ocurrido un error: {error}")
        if connection:
            connection.rollback()
        return 0
    finally:
        if cursor: 
            cursor.close()
        if connection:
            connection.close()

##### **Explicación del Código** 

Esta función permite insertar cualquier `DataFrame` en una tabla de MySQL de manera dinámica y eficiente a través de lo siguientes puntos que, a continuación, se detallan:

1. **Conexión a MySQL**: Se establece la conexión a la base de datos `mercazona` usando las credenciales proporcionadas.
2. **Generación de la consulta SQL**: Se obtienen dinámicamente los nombres de las columnas del `DataFrame` y se construye una consulta `INSERT` con los placeholders (`%s`).
3. **Conversión de datos**: Se transforman los registros del `DataFrame` en tuplas, listas necesarias para `executemany()`.
4. **Ejecución**: `cursor.executemany()` inserta múltiples registros en una sola operación, reduciendo el tiempo de ejecución.
5. **Manejo de errores**: Si ocurre un error, se imprime el mensaje y se revierte la transacción (`rollback()`).
6. **Cierre de recursos**: Finalmente, se cierra el cursor y la conexión para evitar fugas de memoria.

AL igual que en la conexion inicial, se implementa un bloque `try-except` para capturar errores de conexión o ejecución de SQL. Con un mensaje de error en caso de fallo con la descripción del mismo. Se revertería cualquier operación pendiente con `connection.rollback()`.
Por último se cierran las conexiones a través de finally del los objetos cursor y connection.


### 1️⃣2️⃣ **Inserción de las Tablas en MySQL**

Para completar la carga de datos en la base de datos MySQL, se llama a la función `insertar_dataframe()` para insertar los DataFrames generados en sus respectivas tablas.

Este proceso finaliza la carga de los datos en MySQL, asegurando que cada tabla reciba su información correspondiente.


In [30]:
insertar_dataframe(df_tiendas, 'tiendas')
insertar_dataframe(df_empleados, 'empleados')
insertar_dataframe(df_categoria, 'categorias')
insertar_dataframe(df_productos, 'productos')
insertar_dataframe(df_clientes, 'clientes')
insertar_dataframe(df_ordenes, 'ordenes')
insertar_dataframe(df_detalle_orden, 'detalle_orden') 

30000

##  **Conclusión del Proyecto** 

A través de este proyecto se ha demostrado la interoperabilidad entre Python y MySQL Workbench, proporcionando un flujo de trabajo eficiente para la gestión y carga de datos en una base de datos relacional. 

La implementación se divide en dos partes principales:
1. **El esquema SQL**: Definido en un archivo SQL (`MercaZona_schema2.sql`), que establece la estructura de la base de datos.
2. **El Notebook de Python**: Se encarga de la carga del esquema, la generación de datos sintéticos para poblar las tablas y su posterior envío a MySQL.

Gracias a este enfoque, es posible automatizar la creación y administración de bases de datos de manera flexible y escalable. Además, el manejo de errores incorporado garantiza la integridad de los datos y minimiza posibles fallos en la carga.

En resumen, este proyecto es una excelente demostración de cómo **Python y MySQL pueden trabajar juntos para gestionar grandes volúmenes de datos de manera eficiente**. Y si llegaste hasta aquí, felicidades: ahora dominas más sobre bases de datos... o al menos sabes que los cajeros tienen que estar en las órdenes😅. Muchas gracias por tu interés.


# **FIN**